Что делать?

1.Датасет ml-latest

2.Вспомнить подходы, которые мы разбирали

3.Выбрать понравившийся подход к гибридным системам

4.Написать свою

-------

2. Разбирали подходы:

-стекинг

-блендинг

-комбинирование фич

-смешивание

-каскадные рекомендации

3. Выбраны каскадные рекомендации


In [41]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 100)

In [5]:
links = pd.read_csv('ml-latest-small/links.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [6]:
movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [7]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703


In [8]:
links.head(1)

,movieId,imdbId,tmdbId
0,1,114709,862.0


In [9]:
tags.head(1)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994


### объединяем датафреймы

In [10]:
df_rat_mov = ratings.merge(movies, on='movieId', how='left')

In [11]:
df_rat_mov.head(1)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


### преобразуем жанры в числовые признаки

In [12]:
data = df_rat_mov.copy()

In [13]:
data.head(1)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [14]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [15]:
def to_list(s):
    return s.split(' ')

In [16]:
data['genres'] = data['genres'].apply(change_string)

In [17]:
data['genres'] = data['genres'].apply(to_list)

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
one_hot_multiclass = MultiLabelBinarizer()

In [19]:
array_genre = one_hot_multiclass.fit_transform(data['genres'])

In [20]:
array_genre

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 0, 0]])

In [21]:
one_hot_multiclass.classes_

array(['(nogenreslisted)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'SciFi',
       'Thriller', 'War', 'Western'], dtype=object)

In [22]:
df_genre = pd.DataFrame(data=array_genre, columns=['(nogenreslisted)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'FilmNoir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'SciFi',
       'Thriller', 'War', 'Western'])
df_genre.head(1)

,(nogenreslisted),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,IMAX,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [23]:
data_with_genre = data.join(df_genre)

In [24]:
data_with_genre.head(2)

,userId,movieId,rating,timestamp,title,genres,(nogenreslisted),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,IMAX,Musical,Mystery,Romance,SciFi,Thriller,War,Western
0,1,1,4.0,964982703,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [25]:
# data_with_genre.drop(['genres', '(nogenreslisted)'], axis=1, inplace=True)

### tfidf

In [26]:
tfidf_transformer = TfidfTransformer()

In [27]:
movie_genres = [change_string(g) for g in movies.genres]
movie_genres 

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller',
 'Comedy Drama Romance',
 'Comedy Horror',
 'Adventure Animation Children',
 'Drama',
 'Action Adventure Romance',
 'Crime Drama',
 'Drama Romance',
 'Comedy',
 'Comedy',
 'Action Comedy Crime Drama Thriller',
 'Comedy Crime Thriller',
 'Crime Drama Horror Mystery Thriller',
 'Action Crime Thriller',
 'Drama SciFi',
 'Drama Romance',
 'Drama',
 'Children Drama',
 'Drama Romance',
 'Adventure Drama Fantasy Mystery SciFi',
 'Crime Drama',
 'Drama',
 'Mystery SciFi Thriller',
 'Children Drama',
 'Crime Drama',
 'Children Comedy',
 'Comedy Romance',
 'Drama',
 'Drama War',
 'Action Crime Drama',
 'Drama',
 'Action Adventure Fantasy',
 'Comedy Drama Thriller',
 'Drama Romance',
 'Mystery Thriller',
 'Animation Children Drama Musical Romance',
 'Drama R

In [28]:
data_with_genre.sort_values('timestamp', inplace=True)

In [31]:
title_genres = {}

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres

<ipython-input-31-bd73bb19affb>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm_notebook(movies.iterrows()):


In [32]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

In [33]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [35]:
neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=10)

---------

In [42]:
dataset = pd.DataFrame({
    'uid': data_with_genre.userId,
    'iid': data_with_genre.title,
    'rating': data_with_genre.rating
})

In [43]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [44]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [45]:
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

In [46]:
test_pred = algo.test(testset)

In [47]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8665


0.8665014793435794

----------

In [36]:
def recommend_for_user(user_id):
    current_user_id = user_id
    user_movies = data_with_genre[data_with_genre.userId == current_user_id].title.unique()
    
    last_user_movie = user_movies[-1]
    
    movie_genres = title_genres[last_user_movie]
    
    movie_genres = change_string(movie_genres)

    predict = count_vect.transform([movie_genres])
    X_tfidf2 = tfidf_transformer.transform(predict)

    res = neigh.kneighbors(X_tfidf2, return_distance=True)
    
    movies_to_score = movies.iloc[res[1][0]].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
        
    
    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [38]:
data_with_genre[data_with_genre.userId == 2.0].sort_values('rating').head(10)

,userId,movieId,rating,timestamp,title,genres,(nogenreslisted),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,IMAX,Musical,Mystery,Romance,SciFi,Thriller,War,Western
257,2,114060,2.0,1445715276,The Drop (2014),"[Crime, Drama, Thriller]",0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
252,2,91658,2.5,1445714938,"Girl with the Dragon Tattoo, The (2011)","[Drama, Thriller]",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
232,2,318,3.0,1445714835,"Shawshank Redemption, The (1994)","[Crime, Drama]",0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
255,2,109487,3.0,1445715145,Interstellar (2014),"[SciFi, IMAX]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
245,2,77455,3.0,1445714941,Exit Through the Gift Shop (2010),"[Comedy, Documentary]",0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
243,2,71535,3.0,1445714974,Zombieland (2009),"[Action, Comedy, Horror]",0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
251,2,91529,3.5,1445714891,"Dark Knight Rises, The (2012)","[Action, Adventure, Crime, IMAX]",0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
253,2,99114,3.5,1445714874,Django Unchained (2012),"[Action, Drama, Western]",0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
258,2,115713,3.5,1445714854,Ex Machina (2015),"[Drama, SciFi, Thriller]",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
237,2,8798,3.5,1445714960,Collateral (2004),"[Action, Crime, Drama, Thriller]",0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0


In [48]:
recommend_for_user(2.0)

Infernal Affairs (Mou gaan dou) (2002) 4.018141455111026
Nightcrawler (2014) 3.9756515673607713
Simple Plan, A (1998) 3.8617498647737256
Street Kings (2008) 3.5791874519586693
City of Lost Souls, The (Hyôryuu-gai) (2000) 3.5383784664935467
Narc (2002) 3.5375850041995647
Captive, The (2014) 3.517907255095114
Last Seduction, The (1994) 3.5033719341076415
Amateur (1994) 3.4558324009236845


------------------